In [92]:
import sys
sys.path.append('./python-gtmetrix/')
from gtmetrix import GTmetrixInterface
import time
import gspread
from datetime import datetime
from itertools import cycle


credentials_pool = [
    ('uph02154@eoopy.com', 'd6e3713ad0228616517e366aad4db7e9'),
    ('kojohi3627@bcpfm.com', '6d63a4153e3bc9278fa3d1a94b2c140d'),
    ('fajag83615@bcpfm.com', '076c2642acedf4d5481eb8685ef99ec8')
]

gc = gspread.service_account()
sh = gc.open('Black Friday - SW')
worksheet = sh.sheet1


sites = {
    'xkom': 'http://xkom.pl',
    'morele': 'http://morele.pl',
}


In [93]:

def probe(gt, sites):
    for name, url in sites.items():
        my_test = gt.start_test(url)
        yield name, my_test.fetch_results(API_KEY)['results']
        

def save(result, worksheet, row, header=False):
    max_col = chr(ord('A') + len(result))
    keys = list(result.keys())
    values = list(result.values())
    if header:
        worksheet.update(f'A1:{max_col}1', [keys])

    worksheet.update(f'A{row}:{max_col}{row}', [values])
    
    
def run(credentials_pool, sites, worksheet, sleep_in_sec=120):
    row = 2
    header = True
    credentials = cycle(credentials_pool)

    while True:
        email, key = next(credentials)
        print(email)
        gt = GTmetrixInterface(email, key)

        for site, result in probe(gt, sites):
            now = datetime.now().strftime('%H:%M:%S, %d/%m/%Y')
            print(f'{now} - {site} - {email}')
            result['site'] = site
            result['time'] = now
            save(result, worksheet, row, header)
            header = False
            row += 1

        time.sleep(sleep_in_sec)

In [ ]:
run(credentials_pool, sites, worksheet)

uph02154@eoopy.com
19:39:09, 19/11/2020 - xkom - uph02154@eoopy.com
19:39:42, 19/11/2020 - morele - uph02154@eoopy.com
kojohi3627@bcpfm.com
19:42:16, 19/11/2020 - xkom - kojohi3627@bcpfm.com
19:42:49, 19/11/2020 - morele - kojohi3627@bcpfm.com
fajag83615@bcpfm.com
19:45:22, 19/11/2020 - xkom - fajag83615@bcpfm.com
19:45:54, 19/11/2020 - morele - fajag83615@bcpfm.com
uph02154@eoopy.com
19:48:27, 19/11/2020 - xkom - uph02154@eoopy.com
19:49:01, 19/11/2020 - morele - uph02154@eoopy.com
kojohi3627@bcpfm.com
19:51:34, 19/11/2020 - xkom - kojohi3627@bcpfm.com
19:52:07, 19/11/2020 - morele - kojohi3627@bcpfm.com
fajag83615@bcpfm.com
19:54:40, 19/11/2020 - xkom - fajag83615@bcpfm.com
19:55:13, 19/11/2020 - morele - fajag83615@bcpfm.com
uph02154@eoopy.com
19:57:46, 19/11/2020 - xkom - uph02154@eoopy.com
19:58:19, 19/11/2020 - morele - uph02154@eoopy.com
kojohi3627@bcpfm.com
20:00:53, 19/11/2020 - xkom - kojohi3627@bcpfm.com
20:01:25, 19/11/2020 - morele - kojohi3627@bcpfm.com
fajag83615@bcpfm.c

{'spreadsheetId': '1d2cw1E1hEPpsHmIiWKIPoCNppbEW0eVX5GYHar1Qv5g',
 'updatedRange': 'Arkusz1!A1:S1',
 'updatedRows': 1,
 'updatedColumns': 19,
 'updatedCells': 19}

In [41]:
result['results']

{'onload_time': 4707,
 'first_contentful_paint_time': 2323,
 'page_elements': 134,
 'report_url': 'https://gtmetrix.com/reports/xkom.pl/qDKqYTU0/',
 'redirect_duration': 402,
 'first_paint_time': 2323,
 'dom_content_loaded_duration': 109,
 'dom_content_loaded_time': 3141,
 'dom_interactive_time': 3141,
 'page_bytes': 3414948,
 'page_load_time': 4707,
 'html_bytes': 39386,
 'fully_loaded_time': 4892,
 'html_load_time': 1004,
 'rum_speed_index': 4585,
 'yslow_score': 61,
 'pagespeed_score': 74,
 'backend_duration': 259,
 'onload_duration': 0,
 'connect_duration': 343}